In [5]:
import torch
print(torch.__version__)

2.7.1+cpu


In [7]:
#import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import variable

In [13]:
#load and read data from Movielens

#movie information dataset
movies=pd.read_csv('data/datasets/movies.dat',sep='::',header=None,
                   engine='python',encoding='latin-1')

#users information dataset
users=pd.read_csv('data/datasets/users.dat',sep='::',header=None,
                  engine='python',encoding='latin-1')

#ratings information dataset
ratings=pd.read_csv('data/datasets/ratings.dat',sep='::',header=None,
                    engine='python',encoding='latin-1')

In [18]:
movies.head(3)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [20]:
users.head(3)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


In [22]:
ratings.head(3)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [38]:
#trainng and testing datasets

#training set --> user id, movie id, rating, timestamp
training_set=pd.read_csv('data/datasets/ml-100k/u1.base',delimiter='\t',header=None)
#convert pandas dataframe into numpy as array
training_set=np.array(training_set,dtype='int')

#testing set for model evaluation
test_set=pd.read_csv('data/datasets/ml-100k/u1.test',delimiter='\t',header=None)
test_set=np.array(test_set,dtype='int')

In [40]:
training_set

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [42]:
test_set

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [48]:
#get maximum value users and movies from trainign and test set
nb_users=int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies=int(max(max(training_set[:,1]),max(test_set[:,1])))

nb_users,nb_movies

(943, 1682)

In [50]:
#convert data into array with user as row and movie as column
def convert(data):
    new_data=[]
    for id_users in range(1,nb_users+1):
        #movie id rated by users
        id_movies=data[:,1][data[:,0]==id_users]
        #ratings given by the users
        id_ratings=data[:,2][data[:,0]==id_users]

        #set 0 as default when no rating is available
        ratings=np.zeros(nb_movies)
        #movie rating bassed on index value
        ratings[id_movies-1]=id_ratings
        new_data.append(list(ratings))
    print('id_movies:',id_movies)
    print('id_ratings:',id_ratings)
    print('ratings:',ratings)
    return new_data

In [52]:
training_set=convert(training_set)
test_set=convert(test_set)

id_movies: [   2    9   11   12   22   23   24   27   28   31   38   41   42   50
   51   53   54   55   56   58   62   64   67   68   69   72   73   76
   79   80   92   94   96   97   98  100  111  117  121  122  124  127
  132  139  151  161  168  172  173  174  181  182  184  185  186  187
  188  193  194  195  196  200  201  202  204  205  210  215  216  217
  218  219  226  227  228  229  230  231  232  233  234  237  239  274
  281  282  284  318  356  367  373  385  386  391  393  399  401  402
  403  405  406  412  415  419  421  423  426  427  431  443  449  450
  468  470  471  475  485  508  526  541  546  549  559  566  568  569
  570  576  581  585  595  609  614  625  655  672  685  717  720  721
  722  724  732  739  756  763  765  785  794  796  808  816  824  825
  831  840  928  941  943 1011 1028 1044 1047 1067 1074 1188 1228 1330]
id_ratings: [5 3 4 5 4 4 4 4 4 4 3 4 5 4 1 3 4 5 5 4 3 5 4 4 5 2 3 4 5 2 5 4 4 2 5 5 4
 4 3 1 3 5 3 1 4 4 2 4 5 4 4 5 5 2 5 5 4 4 5 4 5 

In [65]:
#convert data into torch sensor

#convert the numpy array into Tensor 
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

In [67]:
#binary conversion of data [1 liked, 0 not liked]

#array index values are the star ratings [0-5]
training_set[training_set==0]=-1
training_set[training_set==1]=0
training_set[training_set==2]=0
training_set[training_set>=3]=1

test_set[test_set==0]=-1
test_set[test_set==1]=0
test_set[test_set==2]=0
test_set[test_set>=3]=1

In [69]:
training_set

tensor([[ 1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])

In [71]:
test_set

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

# RBM Model